In [46]:
#Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#Guardar modelos
from scipy.sparse import csr_matrix
from scipy import sparse


#LDA
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from pprint import pprint

# Carga

## Cargar datos

In [47]:
file_path_meta = "https://raw.githubusercontent.com/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/movies_metadata.csv"

metadata = pd.read_csv(file_path_meta)

metadata.head(1)

,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [48]:
file_path_usuarios = "https://raw.githubusercontent.com/juankquintana/proyecto_recomendacion_peliculas/refs/heads/main/data/base_usuarios.csv"

df_usuarios = pd.read_csv(file_path_usuarios)

df_usuarios = df_usuarios.drop(df_usuarios.columns[0], axis=1)

df_usuarios.head(5)

,userId,movieId,rating
0,123687,163,4.0
1,203854,163,5.0
2,78563,163,1.0
3,203732,163,4.0
4,195546,163,5.0


In [49]:
df_keywords_title = pd.read_pickle('ModelosNLP/df_keywords_title.pkl')
df_keywords_title.head(5)

,id,original_title,keyword_name,cleaned_keywords,genre_names
0,2,Ariel,"[underdog, prison, factory worker, prisoner, h...","[Drama, Crime, underdog, prison, factori worke...","[Drama, Crime]"
1,3,Varjoja paratiisissa,"[salesclerk, helsinki, garbage, independent film]","[Drama, Comedy, salesclerk, helsinki, garbag, ...","[Drama, Comedy]"
2,5,Four Rooms,"[hotel, new year's eve, witch, bet, hotel room...","[Crime, Comedy, hotel, new year eve, witch, be...","[Crime, Comedy]"
3,6,Judgment Night,"[chicago, drug dealer, boxing match, escape, o...","[Action, Thriller, Crime, chicago, drug dealer...","[Action, Thriller, Crime]"
4,11,Star Wars,"[android, galaxy, hermit, death star, lightsab...","[Adventure, Action, Science Fiction, android, ...","[Adventure, Action, Science Fiction]"


In [50]:
df_usuarios['movieId']=df_usuarios['movieId'].astype(str)

df_merge = pd.merge(df_usuarios, df_keywords_title, left_on='movieId', right_on='id', how='inner')

# Seleccionamos las columnas necesarias: userId, movieId, original_title y rating.
df_ratings_peliculas = df_merge[['userId', 'movieId', 'original_title', 'rating']]

# Mostramos el dataframe resultante.
print(df_ratings_peliculas)

         userId movieId                            original_title  rating
0        123687     163                            Ocean's Twelve     4.0
1        203854     163                            Ocean's Twelve     5.0
2         78563     163                            Ocean's Twelve     1.0
3        203732     163                            Ocean's Twelve     4.0
4        195546     163                            Ocean's Twelve     5.0
...         ...     ...                                       ...     ...
2182089  147611   84777                              Gimme an 'F'     3.0
2182090  177150   26964                            Family Viewing     1.5
2182091  243443  169314                  Where The Dead Go to Die     0.5
2182092   45811  126777                            If I Were King     3.0
2182093  228291   84111  Na xie nian, wo men yi qi zhui de nu hai     4.0

[2182094 rows x 4 columns]


In [51]:
#Cargar datos para recomendacion por genero
user_gener_c=pd.read_csv('data/user_gener_c.csv')
user_gener_c = user_gener_c.dropna(subset=['genre'])
user_gener_c.head(5)

,userId,movieId,rating,timestamp,original_title,genre_names,genre_names_exploded,genre
0,123687,163,4.0,1259948252,Ocean's Twelve,"['Thriller', 'Crime']",'Thriller',Thriller
1,123687,163,4.0,1259948252,Ocean's Twelve,"['Thriller', 'Crime']",'Crime',Crime
2,221195,4973,4.0,1465065341,Sous le Sable,"['Drama', 'Mystery']",'Drama',Drama
3,221195,4973,4.0,1465065341,Sous le Sable,"['Drama', 'Mystery']",'Mystery',Mystery
4,134366,1643,4.0,960702344,Ultimo tango a Parigi,"['Drama', 'Romance']",'Drama',Drama


## Cargar funciones de recomendacion

### Cargar Similitud de Coseno con Count Vectorizer

In [52]:
#Toca descargar directo
from scipy import sparse
cosine_sim_sparseCOUNT = sparse.load_npz('ModelosNLP/cosine_sim_sparse_matrixCOUNT.npz')

# Convertirla de nuevo aU densa (si es necesario, aunque puedes seguir usándola como dispersa)
cosine_sim_sparseCOUNT = cosine_sim_sparseCOUNT.toarray()


In [28]:
len(cosine_sim_sparseCOUNT)

30746

In [54]:
df_keywords_title.index

RangeIndex(start=0, stop=30746, step=1)

### Crear otras funciones necesarias:

#### Peliculas ya vistas por el ususario

In [57]:
def rated_movies_by_user(user_id, df=df_ratings_peliculas):
    # Filtrar las películas del usuario especificado
    user_movies = df[df['userId'] == user_id]
    
    # Verificar si el usuario tiene películas registradas
    if user_movies.empty:
        print(f'No se encontraron películas para el usuario {user_id}.')
        return []

    # Ordenar las películas por rating de mayor a menor
    user_movies_sorted = user_movies.sort_values(by='rating', ascending=False)

    # Retornar todos los movieId, título y rating como una lista de tuplas
    return list(zip(user_movies_sorted['movieId'], user_movies_sorted['original_title'], user_movies_sorted['rating']))


#### Top peliculas del usuario

In [58]:
def top_movies_by_user(user_id, top_n=3,df=df_ratings_peliculas):
    # Filtrar las películas del usuario especificado
    user_movies = df[df['userId'] == user_id]
    
    if user_movies.empty:
        print(f'No se encontraron películas para el usuario {user_id}.')
        return []

    # Ordenar las películas por rating de mayor a menor
    user_movies_sorted = user_movies.sort_values(by='rating', ascending=False)
    
    # Seleccionar el top N de películas
    top_movies = user_movies_sorted.head(top_n)
    
    # Retornar el movieId y el rating como una lista de tuplas
    return list(zip(top_movies['movieId'], top_movies['original_title'],top_movies['rating']))


#### Recomendador multiple

In [107]:
def recomendador_multiple(movies_list, cosine_sim=cosine_sim_sparseCOUNT, num_recomendaciones=5, df=df_keywords_title):
    # Lista para almacenar los DataFrames de recomendaciones
    all_recommendations = []

    for movie in movies_list:
        movie_id = movie[0]  # ID de la película
        # Crear una Serie que contiene como índice el 'id' y el valor correspondiente al índice de la película
        indices = pd.Series(df.index, index=df['id'])

        # Verificar si el movie_id está en los índices
        if movie_id not in indices.index:
            print(f'ID de película {movie_id} no encontrado.')
            continue

        # Obtener el índice de la película a partir del movie_id
        idx = indices[movie_id]

        # Obtener el título de la película original
        original_title = df.loc[idx, 'original_title']
        #print(f'Título de la película original: {original_title}')
        #print(f'Índice de la película: {movie_id}')

        # Obtener la similitud de coseno para esa película en particular con todas las películas
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordenar la lista de tuplas por la similitud de coseno en orden descendente
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtener las películas más similares ignorando el primero (que es la misma película)
        sim_scores = sim_scores[1:num_recomendaciones + 1]

        # Obtener los índices y las similitudes
        movie_indices = [i[0] for i in sim_scores]
        sim_scores_values = [i[1] for i in sim_scores]

        # Verificar que los índices obtenidos están dentro del rango del DataFrame
        valid_indices = [idx for idx in movie_indices if idx in df.index]

        valid_id = [df['id'].iloc[idx] for idx in valid_indices]

        # Crear un DataFrame para mostrar los resultados
        result_df = pd.DataFrame({
            'IdPelicula': valid_id,
            'Título': df['original_title'].iloc[valid_indices].values,
            'Similitud Coseno': [sim_scores_values[movie_indices.index(idx)] for idx in valid_indices]
        })

        
        # Añadir el DataFrame de recomendaciones a la lista
        all_recommendations.append(result_df)

    # Concatenar todos los DataFrames en uno solo
    final_recommendations = pd.concat(all_recommendations, ignore_index=True)

    final_recommendations.sort_values(by='Similitud Coseno', ascending=False, inplace=True)
    
    final_recommendations.reset_index(drop=True,inplace=True)

    #final_recommendations=final_recommendations.head(5)
    
    # Imprimir las películas recomendadas
    #print('\nPelículas recomendadas:')
    #print(final_recommendations)

    # Retornar el DataFrame con los resultados
    return final_recommendations.head(5)

In [108]:
lista_usuario=top_movies_by_user(123687)
lista_usuario

[('1967', 'Ask the Dust', 5.0),
 ('2288', 'Closer', 5.0),
 ('97690', 'We Are Legion: The Story of the Hacktivists', 4.5)]

In [109]:
recomendador_multiple(lista_usuario)

,IdPelicula,Título,Similitud Coseno
0,5123,August Rush,0.617161
1,4369,Juste une question d'amour,0.606339
2,235984,Goliyon Ki Raasleela Ram-Leela,0.593366
3,6003,Romeo and Juliet,0.585540
4,27904,แสงศตวรรษ,0.577350


#### Recomendador por genero

In [65]:
def recomendar_peliculas_por_genero():
    # Pedir al usuario que ingrese un género de película
    genero_usuario = input(
        "Por favor, ingrese un género de película que le interese. Las opciones son:\n"
        "'Thriller', 'Crime', 'Drama', 'Mystery', 'Romance', 'Comedy',\n"
        "'Science Fiction', 'Adventure', 'Family', 'Horror', 'History',\n"
        "'TV Movie', 'Fantasy', 'Action', 'Foreign', 'Music',\n"
        "'Documentary', 'Animation', 'Western', 'War'\n"
    ).lower()

    # Filtrar el DataFrame por el género ingresado, aplicando lower para evitar problemas
    df_filtrado = user_gener_c[user_gener_c['genre'].str.lower().str.contains(genero_usuario)]

    # Calcular métricas globales de las películas dentro del género seleccionado
    df_peliculas = df_filtrado.groupby(['movieId', 'original_title']).agg(
        usuarios_unicos=('userId', 'nunique'),   # Contar valores únicos de 'userId'
        rating_pelicula=('rating', 'sum')        # Sumar el rating total
    ).reset_index()

    total_usuarios = df_peliculas['usuarios_unicos'].sum()
    total_rating = df_peliculas['rating_pelicula'].sum()

    # Calcular las probabilidades sin ponderar y ponderada
    df_peliculas['prob_sin_ponderar'] = (df_peliculas['usuarios_unicos'] / total_usuarios)
    df_peliculas['prob_ponderada'] = (df_peliculas['rating_pelicula'] / total_rating)

    # Ordenar películas por cantidad de usuarios (sin ponderar)
    sorted_peliculas_sin_ponderar = df_peliculas.sort_values(by='prob_sin_ponderar', ascending=False)

    # Tomar las primeras 10 películas sin ponderar
    recomendaciones_sin_ponderar = sorted_peliculas_sin_ponderar.head(10)[['original_title', 'prob_sin_ponderar']]

    # Ordenar películas por rating (ponderado)
    sorted_peliculas_ponderada = df_peliculas.sort_values(by='prob_ponderada', ascending=False)

    # Tomar las primeras 10 películas ponderadas
    recomendaciones_ponderada = sorted_peliculas_ponderada.head(10)[['original_title', 'prob_ponderada']]

    # Concatenar ambas listas de recomendaciones
    df_recomendaciones = pd.concat([recomendaciones_ponderada[['original_title']], recomendaciones_sin_ponderar[['original_title']]])

    # Eliminar duplicados de películas
    df_recomendaciones = df_recomendaciones.drop_duplicates(subset='original_title').reset_index(drop=True)

    # Mostrar el mensaje con las recomendaciones
    print(f"Estas son las películas recomendadas en el género '{genero_usuario}', ¡dale una oportunidad!\n")
    return df_recomendaciones



# Recomendador

In [100]:
def recomendacion_personalizada(df_ratings_peliculas=df_ratings_peliculas, df_keywords_title=df_keywords_title, cosine_sim_sparseCOUNT=cosine_sim_sparseCOUNT):
    # Solicitar el ID del usuario
    try:
        user_id = int(input("Por favor, introduce tu ID de usuario: "))
    except ValueError:
        print("ID de usuario no válido. Se recomendarán películas por género.")
        return recomendar_peliculas_por_genero()

    print(f"ID de usuario ingresado: {user_id}")

    # Obtener las películas mejor valoradas por el usuario
    try:
        top_movies = top_movies_by_user(user_id)
        if not top_movies:
            print(f'No hay películas disponibles para el usuario {user_id}. Se recomendarán películas por género.')
            return recomendar_peliculas_por_genero()
    except Exception as e:
        print(f"Error al obtener películas para el usuario {user_id}: {e}. Se recomendarán películas por género.")
        return recomendar_peliculas_por_genero()

    # Obtener las recomendaciones basadas en las mejores películas
    recomendaciones = recomendador_multiple(top_movies, cosine_sim=cosine_sim_sparseCOUNT, df=df_keywords_title)

    # Obtener las películas ya vistas por el usuario
    vistas = rated_movies_by_user(user_id)

    # Filtrar las recomendaciones para eliminar las películas vistas
    vistas_ids = {v[0] for v in vistas}  # Obtener solo los movieId de las películas vistas
    recomendaciones_filtradas = recomendaciones[~recomendaciones['IdPelicula'].isin(vistas_ids)]

    # Imprimir las recomendaciones finales
    print(f'\nUsuario {user_id}. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores:')
    
    # if recomendaciones_filtradas.empty:
    #     print("No hay nuevas recomendaciones disponibles.")
    # else:
    #     print(recomendaciones_filtradas)

    return recomendaciones_filtradas

## Pruebas


In [101]:
recomendacion_personalizada()

ID de usuario ingresado: 6525

Usuario 6525. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores:


,IdPelicula,Título,Similitud Coseno
0,166,La Boum,0.688062
1,252680,Moms' Night Out,0.565217
2,32610,Babes in Arms,0.545545
3,13490,Welcome Home Roscoe Jenkins,0.534522
4,80220,Diabolo menthe,0.531610
5,180721,Blueberry Hill,0.521286
6,356401,Hatuna MeNiyar,0.521286
7,1850,Man on the Moon,0.505076
8,41084,Champagne For Caesar,0.505076
9,258099,Just Before I Go,0.490098


In [102]:
recomendacion_personalizada()

ID de usuario ingresado: 186200

Usuario 186200. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores:


,IdPelicula,Título,Similitud Coseno
0,42744,The Knack... and How to Get It,0.645497
1,6373,"Martha – Meet Frank, Daniel and Laurence",0.527046
2,24077,The Tall Guy,0.527046
3,48764,Procès de Jeanne d'Arc,0.489898
4,26147,FC Venus,0.487950
5,36115,Unmade Beds,0.487950
6,342,Sommersturm,0.434613
7,17097,After Sex,0.430331
8,321,Mambo Italiano,0.389249
9,112160,Chroniques sexuelles d'une famille d'aujourd'hui,0.378630


In [103]:
recomendacion_personalizada()

ID de usuario ingresado: 270832

Usuario 270832. Aquí tienes unas recomendaciones de películas que aún no has visto según tus ratings anteriores:


,IdPelicula,Título,Similitud Coseno
0,11595,Another 48 Hrs.,0.588348
1,6723,The Rookie,0.566139
2,72002,The Last Mile,0.554700
3,245527,Convict,0.554700
4,17466,Death Warrant,0.526235
5,26465,Big Pun: The Legacy,0.478091
6,30586,Biggie and Tupac,0.478091
7,59333,Official Rejection,0.462910
8,87851,New Muslim Cool,0.462910
9,21724,Dance Flick,0.445435
